In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.env.dunderbot_env import DunderBotEnv

# TODO:
- remove the bias when getting the observations (suggestion in the 
- exchange data to something more interesting, consider implementing time steps
- modularize actions, rewards, and observations
- understand the rendering, especially during learning
- understand the time steps in learn and predict
- add config

# Run main

In [17]:
import gym
import json
import datetime as dt

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

In [18]:
from src.env.dunderbot_env import DunderBotEnv

import pandas as pd

df = pd.read_csv('./data/AAPL.csv')
df = df.sort_values('Date')
df = df.drop(columns=['Unnamed: 0'])

In [19]:
# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: DunderBotEnv(df)])

model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=20000)

/code/src/env/dunderbot_env.py:77: RuntimeWarning: invalid value encountered in double_scalars
  prev_cost + additional_cost) / (self.shares_held + shares_bought)


--------------------------------------
| approxkl           | 1.0256373e-07 |
| clipfrac           | 0.0           |
| explained_variance | 3.58e-07      |
| fps                | 195           |
| n_updates          | 1             |
| policy_entropy     | 2.8380976     |
| policy_loss        | -8.389156e-05 |
| serial_timesteps   | 128           |
| time_elapsed       | 3.15e-05      |
| total_timesteps    | 128           |
| value_loss         | 25131590.0    |
--------------------------------------
--------------------------------------
| approxkl           | 1.0720984e-07 |
| clipfrac           | 0.0           |
| explained_variance | -1.07e-06     |
| fps                | 344           |
| n_updates          | 2             |
| policy_entropy     | 2.83845       |
| policy_loss        | -3.378978e-05 |
| serial_timesteps   | 256           |
| time_elapsed       | 0.656         |
| total_timesteps    | 256           |
| value_loss         | 16192970.0    |
-------------------------

--------------------------------------
| approxkl           | 8.2662256e-05 |
| clipfrac           | 0.0           |
| explained_variance | 2.35e-05      |
| fps                | 296           |
| n_updates          | 18            |
| policy_entropy     | 2.866619      |
| policy_loss        | -0.0026157063 |
| serial_timesteps   | 2304          |
| time_elapsed       | 7.26          |
| total_timesteps    | 2304          |
| value_loss         | 2649938.5     |
--------------------------------------
--------------------------------------
| approxkl           | 0.0004444363  |
| clipfrac           | 0.001953125   |
| explained_variance | -5.77e-05     |
| fps                | 334           |
| n_updates          | 19            |
| policy_entropy     | 2.867098      |
| policy_loss        | -0.0004201762 |
| serial_timesteps   | 2432          |
| time_elapsed       | 7.69          |
| total_timesteps    | 2432          |
| value_loss         | 3856722.8     |
-------------------------

--------------------------------------
| approxkl           | 0.0016015088  |
| clipfrac           | 0.001953125   |
| explained_variance | 2.35e-05      |
| fps                | 342           |
| n_updates          | 35            |
| policy_entropy     | 2.8814516     |
| policy_loss        | 0.00074208016 |
| serial_timesteps   | 4480          |
| time_elapsed       | 13.8          |
| total_timesteps    | 4480          |
| value_loss         | 1782599200.0  |
--------------------------------------
--------------------------------------
| approxkl           | 0.0003460435  |
| clipfrac           | 0.0           |
| explained_variance | -9.54e-07     |
| fps                | 337           |
| n_updates          | 36            |
| policy_entropy     | 2.8818638     |
| policy_loss        | -0.0040540863 |
| serial_timesteps   | 4608          |
| time_elapsed       | 14.2          |
| total_timesteps    | 4608          |
| value_loss         | 781958850.0   |
-------------------------

-------------------------------------
| approxkl           | 0.0014764587 |
| clipfrac           | 0.0          |
| explained_variance | -2.38e-06    |
| fps                | 347          |
| n_updates          | 52           |
| policy_entropy     | 2.869822     |
| policy_loss        | 0.0016819716 |
| serial_timesteps   | 6656         |
| time_elapsed       | 20.2         |
| total_timesteps    | 6656         |
| value_loss         | 167229490.0  |
-------------------------------------
---------------------------------------
| approxkl           | 0.0002294808   |
| clipfrac           | 0.0            |
| explained_variance | 1.62e-05       |
| fps                | 331            |
| n_updates          | 53             |
| policy_entropy     | 2.868599       |
| policy_loss        | -0.00030095922 |
| serial_timesteps   | 6784           |
| time_elapsed       | 20.6           |
| total_timesteps    | 6784           |
| value_loss         | 345439500.0    |
--------------------------

...

---------------------------------------
| approxkl           | 0.00035081626  |
| clipfrac           | 0.0            |
| explained_variance | 2.15e-06       |
| fps                | 254            |
| n_updates          | 85             |
| policy_entropy     | 2.8627439      |
| policy_loss        | 0.0010303357   |
| serial_timesteps   | 10880          |
| time_elapsed       | 34.3           |
| total_timesteps    | 10880          |
| value_loss         | 204169610000.0 |
---------------------------------------
-------------------------------------
| approxkl           | 0.0001806106 |
| clipfrac           | 0.0          |
| explained_variance | 1.22e-05     |
| fps                | 321          |
| n_updates          | 86           |
| policy_entropy     | 2.863        |
| policy_loss        | 0.0025121218 |
| serial_timesteps   | 11008        |
| time_elapsed       | 34.8         |
| total_timesteps    | 11008        |
| value_loss         | 143134540.0  |
------------------------

--------------------------------------
| approxkl           | 0.00010850059 |
| clipfrac           | 0.0           |
| explained_variance | -3.58e-07     |
| fps                | 288           |
| n_updates          | 102           |
| policy_entropy     | 2.8555741     |
| policy_loss        | 3.1441683e-05 |
| serial_timesteps   | 13056         |
| time_elapsed       | 41.8          |
| total_timesteps    | 13056         |
| value_loss         | 42207007000.0 |
--------------------------------------
---------------------------------------
| approxkl           | 0.0009200489   |
| clipfrac           | 0.0            |
| explained_variance | 0              |
| fps                | 273            |
| n_updates          | 103            |
| policy_entropy     | 2.8557549      |
| policy_loss        | -0.00015716703 |
| serial_timesteps   | 13184          |
| time_elapsed       | 42.2           |
| total_timesteps    | 13184          |
| value_loss         | 54377095000.0  |
-------------

---------------------------------------
| approxkl           | 0.0008660609   |
| clipfrac           | 0.0            |
| explained_variance | -1.19e-07      |
| fps                | 317            |
| n_updates          | 118            |
| policy_entropy     | 2.8572254      |
| policy_loss        | -0.0005458263  |
| serial_timesteps   | 15104          |
| time_elapsed       | 48.2           |
| total_timesteps    | 15104          |
| value_loss         | 169819540000.0 |
---------------------------------------
---------------------------------------
| approxkl           | 0.00034588715  |
| clipfrac           | 0.0            |
| explained_variance | 0              |
| fps                | 346            |
| n_updates          | 119            |
| policy_entropy     | 2.8573935      |
| policy_loss        | -0.0035431385  |
| serial_timesteps   | 15232          |
| time_elapsed       | 48.6           |
| total_timesteps    | 15232          |
| value_loss         | 214778250000.0 |


--------------------------------------
| approxkl           | 0.00022301328 |
| clipfrac           | 0.0           |
| explained_variance | -2.38e-07     |
| fps                | 327           |
| n_updates          | 134           |
| policy_entropy     | 2.8635952     |
| policy_loss        | 0.0010663371  |
| serial_timesteps   | 17152         |
| time_elapsed       | 54.3          |
| total_timesteps    | 17152         |
| value_loss         | 9590136000.0  |
--------------------------------------
--------------------------------------
| approxkl           | 0.00010837106 |
| clipfrac           | 0.0           |
| explained_variance | 2.98e-07      |
| fps                | 330           |
| n_updates          | 135           |
| policy_entropy     | 2.8644133     |
| policy_loss        | 0.0010435618  |
| serial_timesteps   | 17280         |
| time_elapsed       | 54.7          |
| total_timesteps    | 17280         |
| value_loss         | 17679587000.0 |
-------------------------

---------------------------------------
| approxkl           | 6.7588365e-07  |
| clipfrac           | 0.0            |
| explained_variance | -1.19e-07      |
| fps                | 321            |
| n_updates          | 150            |
| policy_entropy     | 2.861279       |
| policy_loss        | -3.5895966e-05 |
| serial_timesteps   | 19200          |
| time_elapsed       | 60.5           |
| total_timesteps    | 19200          |
| value_loss         | 662698850000.0 |
---------------------------------------
---------------------------------------
| approxkl           | 0.00089989     |
| clipfrac           | 0.0            |
| explained_variance | 0              |
| fps                | 326            |
| n_updates          | 151            |
| policy_entropy     | 2.8616157      |
| policy_loss        | 0.004586609    |
| serial_timesteps   | 19328          |
| time_elapsed       | 60.9           |
| total_timesteps    | 19328          |
| value_loss         | 719522200000.0 |


In [21]:
obs = env.reset()
for i in range(2000):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()

Step: 4235
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 4236
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 4237
Balance: 10000
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 4238
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.09769813647104 (Total sales value: 0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 4239
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.09769813647104 (Total sales value: 0)
Net worth: 9980.800278810435 (Max net worth: 10000)
Profit: -19.199721189565025
Step: 4240
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.0976

Step: 4283
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.09769813647104 (Total sales value: 0.0)
Net worth: 10314.307510165194 (Max net worth: 10907.760106039936)
Profit: 314.3075101651939
Step: 4284
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.09769813647104 (Total sales value: 0.0)
Net worth: 10151.179331061328 (Max net worth: 10907.760106039936)
Profit: 151.1793310613284
Step: 4285
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.09769813647104 (Total sales value: 0.0)
Net worth: 10191.121042917748 (Max net worth: 10907.760106039936)
Profit: 191.12104291774813
Step: 4286
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.09769813647104 (Total sales value: 0.0)
Net worth: 10016.290254294168 (Max net worth: 10907.760106039936)
Profit: 16.290254294168335
Step: 4287
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)


Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.09769813647104 (Total sales value: 0.0)
Net worth: 11648.611609766473 (Max net worth: 12105.604269178599)
Profit: 1648.6116097664726
Step: 4334
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.09769813647104 (Total sales value: 0.0)
Net worth: 11465.738666829715 (Max net worth: 12105.604269178599)
Profit: 1465.738666829715
Step: 4335
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.09769813647104 (Total sales value: 0.0)
Net worth: 11358.860311832657 (Max net worth: 12105.604269178599)
Profit: 1358.8603118326573
Step: 4336
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost for held shares: 109.09769813647104 (Total sales value: 0.0)
Net worth: 11351.829847804964 (Max net worth: 12105.604269178599)
Profit: 1351.8298478049637
Step: 4337
Balance: 72.10946958113527
Shares held: 91 (Total sold: 0)
Avg cost f

Step: 4381
Balance: 9730.08525903778
Shares held: 15 (Total sold: 76)
Avg cost for held shares: 109.09769813647104 (Total sales value: 9657.975789456645)
Net worth: 11678.44520893292 (Max net worth: 12105.604269178599)
Profit: 1678.4452089329207
Step: 4382
Balance: 9730.08525903778
Shares held: 15 (Total sold: 76)
Avg cost for held shares: 109.09769813647104 (Total sales value: 9657.975789456645)
Net worth: 11683.072939117139 (Max net worth: 12105.604269178599)
Profit: 1683.072939117139
Step: 4383
Balance: 9730.08525903778
Shares held: 15 (Total sold: 76)
Avg cost for held shares: 109.09769813647104 (Total sales value: 9657.975789456645)
Net worth: 11671.471123739499 (Max net worth: 12105.604269178599)
Profit: 1671.4711237394986
Step: 4384
Balance: 9730.08525903778
Shares held: 15 (Total sold: 76)
Avg cost for held shares: 109.09769813647104 (Total sales value: 9657.975789456645)
Net worth: 11662.254523904858 (Max net worth: 12105.604269178599)
Profit: 1662.2545239048577
Step: 4385
Bal

Step: 4423
Balance: 10235.810040959095
Shares held: 11 (Total sold: 80)
Avg cost for held shares: 109.09769813647104 (Total sales value: 10163.70057137796)
Net worth: 11572.233301713837 (Max net worth: 12105.604269178599)
Profit: 1572.2333017138371
Step: 4424
Balance: 10235.810040959095
Shares held: 11 (Total sold: 80)
Avg cost for held shares: 109.09769813647104 (Total sales value: 10163.70057137796)
Net worth: 11547.230328844336 (Max net worth: 12105.604269178599)
Profit: 1547.2303288443363
Step: 4425
Balance: 10235.810040959095
Shares held: 11 (Total sold: 80)
Avg cost for held shares: 109.09769813647104 (Total sales value: 10163.70057137796)
Net worth: 11523.967932950569 (Max net worth: 12105.604269178599)
Profit: 1523.9679329505689
Step: 4426
Balance: 10235.810040959095
Shares held: 11 (Total sold: 80)
Avg cost for held shares: 109.09769813647104 (Total sales value: 10163.70057137796)
Net worth: 11481.017092847236 (Max net worth: 12105.604269178599)
Profit: 1481.0170928472362
Step

...

Step: 799
Balance: 15034.964959018493
Shares held: 43 (Total sold: 2779)
Avg cost for held shares: 14.788347356167854 (Total sales value: 92335.85600459804)
Net worth: 15890.517694378159 (Max net worth: 16815.40730239786)
Profit: 5890.517694378159
Step: 800
Balance: 15034.964959018493
Shares held: 43 (Total sold: 2779)
Avg cost for held shares: 14.788347356167854 (Total sales value: 92335.85600459804)
Net worth: 15951.284315421282 (Max net worth: 16815.40730239786)
Profit: 5951.284315421282
Step: 801
Balance: 15034.964959018493
Shares held: 43 (Total sold: 2779)
Avg cost for held shares: 14.788347356167854 (Total sales value: 92335.85600459804)
Net worth: 15949.113356026284 (Max net worth: 16815.40730239786)
Profit: 5949.113356026284
Step: 802
Balance: 15034.964959018493
Shares held: 43 (Total sold: 2779)
Avg cost for held shares: 14.788347356167854 (Total sales value: 92335.85600459804)
Net worth: 15924.910568753847 (Max net worth: 16815.40730239786)
Profit: 5924.910568753847
Step: 80

Step: 837
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 838
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 839
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 840
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 841
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost

Profit: 5905.516707700703
Step: 875
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 876
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 877
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 878
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 879
Balance: 15905.516707700703
Shares held: 0 (

Step: 914
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 915
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 916
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 917
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost for held shares: 0 (Total sales value: 93206.40775328025)
Net worth: 15905.516707700703 (Max net worth: 16815.40730239786)
Profit: 5905.516707700703
Step: 918
Balance: 15905.516707700703
Shares held: 0 (Total sold: 2822)
Avg cost

Balance: 14769.968504391174
Shares held: 0 (Total sold: 3810)
Avg cost for held shares: 0 (Total sales value: 107970.30506177155)
Net worth: 14769.968504391174 (Max net worth: 16815.40730239786)
Profit: 4769.968504391174
Step: 954
Balance: 14248.030083459953
Shares held: 29 (Total sold: 3810)
Avg cost for held shares: 17.9978765838352 (Total sales value: 107970.30506177155)
Net worth: 14769.968504391174 (Max net worth: 16815.40730239786)
Profit: 4769.968504391174
Step: 955
Balance: 14248.030083459953
Shares held: 29 (Total sold: 3810)
Avg cost for held shares: 17.9978765838352 (Total sales value: 107970.30506177155)
Net worth: 14771.596749979815 (Max net worth: 16815.40730239786)
Profit: 4771.596749979815
Step: 956
Balance: 14796.04272419273
Shares held: 0 (Total sold: 3839)
Avg cost for held shares: 0 (Total sales value: 108518.31770250433)
Net worth: 14796.04272419273 (Max net worth: 16815.40730239786)
Profit: 4796.04272419273
Step: 957
Balance: 14796.04272419273
Shares held: 0 (Tota